# Overlapとmp_term_nameの情報を統合する

In [1]:
# Move up to top directory
import os
os.getcwd()
os.chdir('../')
os.getcwd()

'/mnt/g/impc-genes_with_similar_phenotypes'

In [2]:
from pathlib import Path
from collections import defaultdict
from itertools import combinations
import csv
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
path_overlap = Path("data", "overlap_ratios.csv")
path_mp = Path("data", "mp_term_name", "male_infertility.csv")

In [4]:
df_overlap = pd.read_csv(path_overlap)
df_mp = pd.read_csv(path_mp)

In [5]:
df_mp = df_mp[["marker_symbol", "effect_size"]].drop_duplicates()
df_mp

,marker_symbol,effect_size
0,Tex38,1
1,Pou2af3,1
2,Stoml2,1
3,Sbf1,1
4,Cyrib,1
...,...,...
320,Cep19,1
321,Bzw1,1
322,Septin10,1
323,Cfap57,1


In [6]:
# print(df_overlap[df_overlap["intersect_count"] > 5 & df_overlap["overlap_ratio"] > 0.5])

In [42]:
# df_overlap_filterd = df_overlap[df_overlap["overlapped_mp_number"] > 8]
df_overlap_filterd = df_overlap[(df_overlap["overlap_ratio"] > 0.5) & (df_overlap["overlapped_mp_number"] > 2)]

In [43]:
df_filtered = df_overlap_filterd[
    (df_overlap_filterd['marker1'].isin(df_mp['marker_symbol'])) &
    (df_overlap_filterd['marker2'].isin(df_mp['marker_symbol']))
]

In [44]:
df_merged = pd.merge(df_filtered, df_mp, how='inner', left_on='marker1', right_on='marker_symbol')

df_merged.head(3)

,marker1,marker2,overlap_ratio,overlapped_mp_number,overlapped_mp,marker1_mp,marker2_mp,marker_symbol,effect_size
0,Dnmt3l,Cntd1,0.600,3,"{'female infertility', 'small testis', 'male i...","{'increased mean corpuscular hemoglobin', 'fem...","{'female infertility', 'small testis', 'male i...",Dnmt3l,1
1,Nphp1,Pgap6,0.600,3,"{'small testis', 'abnormal skin morphology', '...","{'abnormal uterus morphology', 'male infertili...","{'male infertility', 'small testis', 'abnormal...",Nphp1,1
2,Spidr,Kash5,0.545,6,"{'increased total body fat amount', 'decreased...","{'increased total body fat amount', 'decreased...","{'increased total body fat amount', 'decreased...",Spidr,1


In [45]:
df_network = df_merged[["marker1", "marker2", "effect_size", "overlap_ratio"]]

df_network.columns = ["from", "to", "node_size", "edge_size"]
print(len(df_network))
print(df_network.head(3))


32
     from     to  node_size  edge_size
0  Dnmt3l  Cntd1          1      0.600
1   Nphp1  Pgap6          1      0.600
2   Spidr  Kash5          1      0.545


In [46]:
df_network.to_csv(Path("data", "network.csv"), index=False)

In [41]:
# %%bash

# python scripts/vis_network.py